# Materie Prime

##### setup

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf

username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
             complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
         complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
         complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
         complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
            pathPart1 = "/Users/"
            pathPart2 = "/documents/Github/SCGProject/Datasets/"
            complePath = pathPart1+username+pathPart2

            dfCambio = pd.read_csv(
                complePath+"/CorrectedDatasets/dfCambio.csv")

            dfClienti = pd.read_csv(
                complePath+"/CorrectedDatasets/dfClienti.csv")

            dfConsumi = pd.read_csv(
                complePath + "/CorrectedDatasets/dfConsumi.csv")

            dfCostoOrarioConsuntivo = pd.read_csv(
                complePath+"/CorrectedDatasets/dfConsuntivo.csv")

            dfCostoOrarioBudget = pd.read_csv(
                complePath+"CorrectedDatasets/dfCostoOrario.csv")

            dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

            dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

            dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")


Username:  davidguzman


## 1) Scostamento costo totale Materia Prima

In [3]:
dfConsumi.rename(columns = {'Budget/cons':'budget'}, inplace = True)
consumiConsuntivo = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'CONSUNTIVO'")
consumiBudget = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'BUDGET'")
sumConsumiConsuntivo = sqldf('select sum(ImportoCostoTOTALE) from consumiConsuntivo')
sumConsumiBudget = sqldf('select sum(ImportoCostoTOTALE) from consumiBudget')
sumConsumiBudget-sumConsumiConsuntivo

,sum(ImportoCostoTOTALE)
0,-96733.03


## 2) Scostamento costo totale per CodiceMP

In [ ]:
# quantità totale di codici MP 
countConsumiMPc = sqldf('select distinct CodiceMP, count(*) from consumiConsuntivo group by CodiceMP order by count(*) desc')
countConsumiMPb = sqldf('select distinct CodiceMP, count(*) from consumiBudget group by CodiceMP order by count(*) desc')
print("Consuntivo")
print(countConsumiMPc)
print("Budget")
print(countConsumiMPb)
#Considerazione= Consuntivo ha più codici rispetto a budget
# 629-553 = 76 codici single

In [ ]:
consumiConsuntivoByMP = sqldf('select codiceMP, sum(importoCostoTOTALE) as sumCostoMP from consumiConsuntivo group by codiceMP order by sumCostoMP desc')
consumiBudgetByMP = sqldf('select codiceMP, sum(importoCostoTOTALE) as sumCostoMP from consumiBudget group by codiceMP order by sumCostoMP desc')

In [ ]:
#Join e differenza tra le 2 table precedenti
scostamentoConsumiMP = sqldf('''select c.CodiceMP, b.sumCostoMP as sumCostoMPBudget, c.sumCostoMP as sumCostoMPConsuntivo, b.sumCostoMP-c.sumCostoMP as scostamento 
from consumiConsuntivoByMP as c join consumiBudgetByMP as b 
on c.codiceMP = b.codiceMP
order by scostamento desc''')

scostamentoConsumiMP

In [ ]:
sqldf('select sum (scostamento) from scostamentoConsumiMP') #Considero solo quelli che sono presenti in entrambi
# notiamo che lo scostamento totale continua essere in linea con quello orginale, 
# la differenza potrebbe essere giustificata da MP senza con budget e senza consuntivo o viceversa

#### Versione con full join (not supported by SQL)

In [ ]:
#scostamentoConsumiMP = sqldf('''select c.CodiceMP, b.sumCostoMP as sumCostoMPBudget, c.sumCostoMP as sumCostoMPConsuntivo, b.sumCostoMP-c.sumCostoMP as scostamento 
#from consumiConsuntivoByMP as c full join consumiBudgetByMP as b 
#on c.codiceMP = b.codiceMP
#order by scostamento desc''')

#scostamentoConsumiMP

### scostamento di impiego e prezzo


In [ ]:
consumiConsuntivoUnit = sqldf('''select distinct NrMovimento, CodiceMP, NrArticolo, QuantityMPImpiegata, ImportoCostoTOTALE/QuantityMPImpiegata as CostoUnitario, ImportoCostoTOTALE 
from consumiConsuntivo
order by CodiceMP''')
consumiBudgetUnit = sqldf('''select distinct NrMovimento, CodiceMP, NrArticolo, QuantityMPImpiegata, ImportoCostoTOTALE/QuantityMPImpiegata as CostoUnitario, ImportoCostoTOTALE 
from consumiBudget
order by CodiceMP''')

In [ ]:
consumiConsuntivoUnitByMP = sqldf('''select distinct CodiceMP, sum(QuantityMPImpiegata) as sumQuantityMPImpiegata, sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario, sum(ImportoCostoTOTALE) as sumCostoMP
from consumiConsuntivo
group by CodiceMP
order by CodiceMP''')
consumiBudgetUnitByMP = sqldf('''select distinct CodiceMP, sum(QuantityMPImpiegata) as sumQuantityMPImpiegata, sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario, sum(ImportoCostoTOTALE) as sumCostoMP
from consumiBudget
group by CodiceMP
order by CodiceMP''')

#### scostamento di impiego

In [ ]:
scostamentoConsumiMPImpiego = sqldf('''select c.CodiceMP, b.sumCostoMP as ImpiegoStandard, c.sumQuantityMPImpiegata*b.CostoUnitario as ImpiegoEffettivo, b.sumCostoMP-c.sumQuantityMPImpiegata*b.CostoUnitario as scostamento
from consumiConsuntivoUnitByMP as c join consumiBudgetUnitByMP as b 
on c.CodiceMP = b.CodiceMP
order by c.CodiceMP''')
scostamentoConsumiMPImpiego

,CodiceMP,ImpiegoStandard,ImpiegoEffettivo,scostamento
0,ART0001073,2.59,25.554667,-22.964667
1,ART0001074,203.48,253.791932,-50.311932
2,ART0001076,158.70,844.905000,-686.205000
3,ART0001078,5.47,47.295741,-41.825741
4,ART0001080,1.21,2.634677,-1.424677
...,...,...,...,...
466,ART0005934,25.83,599.966667,-574.136667
467,ART0005944,0.00,0.000000,0.000000
468,ART0005951,148.05,1035.504000,-887.454000
469,ART0005953,826.57,28.502414,798.067586


In [ ]:
tempImpiego = sqldf('select sum(scostamento) from scostamentoConsumiMPImpiego')
tempImpiego

,sum(scostamento)
0,-91732.950316


#### scostamento di prezzo

In [ ]:
scostamentoConsumiMPPrezzo = sqldf('''select c.CodiceMP, c.sumQuantityMPImpiegata*b.CostoUnitario as ImpiegoEffettivo, c.sumCostoMP as BudgetEffettivo, c.sumQuantityMPImpiegata*b.CostoUnitario-c.sumCostoMP as scostamento
from consumiConsuntivoUnitByMP as c join consumiBudgetUnitByMP as b 
on c.CodiceMP = b.CodiceMP
order by c.CodiceMP''')
scostamentoConsumiMPPrezzo 

,CodiceMP,ImpiegoEffettivo,BudgetEffettivo,scostamento
0,ART0001073,25.554667,25.01,0.544667
1,ART0001074,253.791932,257.05,-3.258068
2,ART0001076,844.905000,890.51,-45.605000
3,ART0001078,47.295741,67.26,-19.964259
4,ART0001080,2.634677,2.97,-0.335323
...,...,...,...,...
466,ART0005934,599.966667,649.49,-49.523333
467,ART0005944,0.000000,0.00,0.000000
468,ART0005951,1035.504000,1004.04,31.464000
469,ART0005953,28.502414,42.40,-13.897586


In [ ]:
tempPrezzo = sqldf('select sum(scostamento) from scostamentoConsumiMPPrezzo')
tempPrezzo

,sum(scostamento)
0,1266.620316


In [ ]:
tempImpiego+tempPrezzo # scostamento totale resta uguale, TODO full join

,sum(scostamento)
0,-90466.33


## 3) Scostamento costo totale per articolo

In [ ]:
# quantità totale di articoli
countConsumiMPc = sqldf('select distinct NrArticolo, count(*) from consumiConsuntivo group by NrArticolo order by count(*) desc')
countConsumiMPb = sqldf('select distinct NrArticolo, count(*) from consumiBudget group by NrArticolo order by count(*) desc')
print(countConsumiMPc)
print(countConsumiMPb) #Stesso numero di articoli

     NrArticolo  count(*)
0    ART0005136       234
1    ART0000782       184
2    ART0000752       133
3    ART0000847       128
4    ART0000748       119
..          ...       ...
499  ART0000115         1
500  ART0000043         1
501  ART0000042         1
502  ART0000041         1
503  ART0000018         1

[504 rows x 2 columns]
     NrArticolo  count(*)
0    ART0000795        66
1    ART0000762        66
2    ART0000782        64
3    ART0004099        52
4    ART0000752        49
..          ...       ...
499  ART0000119         1
500  ART0000043         1
501  ART0000042         1
502  ART0000041         1
503  ART0000018         1

[504 rows x 2 columns]


In [ ]:
consumiConsuntivoByArt = sqldf('select NrArticolo, sum(importoCostoTOTALE) as sumCostoMP from consumiConsuntivo group by NrArticolo order by sumCostoMP desc')
consumiBudgetByArt = sqldf('select NrArticolo, sum(importoCostoTOTALE) as sumCostoMP from consumiBudget group by NrArticolo order by sumCostoMP desc')

In [ ]:
scostamentoConsumiArt = sqldf('''select c.NrArticolo, b.sumCostoMP as sumCostoMPBudget, c.sumCostoMP as sumCostoMPConsuntivo, b.sumCostoMP-c.sumCostoMP as scostamento 
from consumiConsuntivoByArt as c join consumiBudgetByArt as b 
on c.NrArticolo = b.NrArticolo
order by scostamento desc''')

scostamentoConsumiArt

,NrArticolo,sumCostoMPBudget,sumCostoMPConsuntivo,scostamento
0,ART0000762,15123.70,3111.43,12012.27
1,ART0000826,3913.21,651.27,3261.94
2,ART0000812,3900.33,1246.50,2653.83
3,ART0004146,1790.92,605.97,1184.95
4,ART0001913,1469.78,524.13,945.65
...,...,...,...,...
499,ART0000752,6489.77,12505.88,-6016.11
500,ART0000811,3343.14,10540.89,-7197.75
501,ART0000853,11467.17,18994.86,-7527.69
502,ART0000847,706.68,15023.61,-14316.93


In [ ]:
sqldf('select sum (scostamento) from scostamentoConsumiArt') 
# notiamo che lo scostamento totale è completamente in linea con quello orginale 

,sum (scostamento)
0,-96733.03


### scostamento di impiego e prezzo

In [ ]:
consumiConsuntivoUnit = sqldf('''select distinct NrMovimento, CodiceMP, NrArticolo, QuantityMPImpiegata, ImportoCostoTOTALE/QuantityMPImpiegata as CostoUnitario, ImportoCostoTOTALE 
from consumiConsuntivo
order by CodiceMP''')
consumiBudgetUnit = sqldf('''select distinct NrMovimento, CodiceMP, NrArticolo, QuantityMPImpiegata, ImportoCostoTOTALE/QuantityMPImpiegata as CostoUnitario, ImportoCostoTOTALE 
from consumiBudget
order by CodiceMP''')

In [ ]:
# sqldf('select NrArticolo,sum(QuantityMPImpiegata) as sumQuantityMPImpiegata, sum(CostoUnitario) as sumCostoUnitario, sum(ImportoCostoTOTALE) as sumCostoMP from consumiConsuntivoUnit group by NrArticolo')
consumiConsuntivoUnitByArt = sqldf('''select distinct NrArticolo, sum(QuantityMPImpiegata) as sumQuantityMPImpiegata, sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario, sum(ImportoCostoTOTALE) as sumCostoMP
from consumiConsuntivo
group by NrArticolo
order by NrArticolo''')
consumiBudgetUnitByArt = sqldf('''select distinct NrArticolo, sum(QuantityMPImpiegata) as sumQuantityMPImpiegata, sum(ImportoCostoTOTALE)/sum(QuantityMPImpiegata) as CostoUnitario, sum(ImportoCostoTOTALE) as sumCostoMP
from consumiBudget
group by NrArticolo
order by NrArticolo''')

#### scostamento di impiego

In [ ]:
scostamentoConsumiArtImpiego = sqldf('''select c.NrArticolo, b.sumCostoMP as ImpiegoStandard, c.sumQuantityMPImpiegata*b.CostoUnitario as ImpiegoEffettivo, b.sumCostoMP-c.sumQuantityMPImpiegata*b.CostoUnitario as scostamento
from consumiConsuntivoUnitByArt as c join consumiBudgetUnitByArt as b 
on c.NrArticolo = b.NrArticolo
order by c.NrArticolo''')
scostamentoConsumiArtImpiego


,NrArticolo,ImpiegoStandard,ImpiegoEffettivo,scostamento
0,ART0000018,5.15,5.150000,0.000000
1,ART0000041,92.00,92.000000,0.000000
2,ART0000042,104.00,104.000000,0.000000
3,ART0000043,36.06,35.716571,0.343429
4,ART0000115,434.44,216.720644,217.719356
...,...,...,...,...
499,ART0005109,6.97,15.891600,-8.921600
500,ART0005110,74.00,407.000000,-333.000000
501,ART0005111,16.00,64.000000,-48.000000
502,ART0005112,26.00,104.000000,-78.000000


In [ ]:
tempImpiego = sqldf('select sum(scostamento) from scostamentoConsumiArtImpiego')
tempImpiego

,sum(scostamento)
0,-337819.267312


#### scostamento di prezzo

In [ ]:
scostamentoConsumiArtPrezzo = sqldf('''select c.NrArticolo, c.sumQuantityMPImpiegata*b.CostoUnitario as ImpiegoEffettivo, c.sumCostoMP as BudgetEffettivo, c.sumQuantityMPImpiegata*b.CostoUnitario-c.sumCostoMP as scostamento
from consumiConsuntivoUnitByArt as c join consumiBudgetUnitByArt as b 
on c.NrArticolo = b.NrArticolo
order by c.NrArticolo''')
scostamentoConsumiArtPrezzo 

,NrArticolo,ImpiegoEffettivo,BudgetEffettivo,scostamento
0,ART0000018,5.150000,5.15,0.000000
1,ART0000041,92.000000,88.00,4.000000
2,ART0000042,104.000000,100.00,4.000000
3,ART0000043,35.716571,35.71,0.006571
4,ART0000115,216.720644,268.26,-51.539356
...,...,...,...,...
499,ART0005109,15.891600,14.74,1.151600
500,ART0005110,407.000000,304.00,103.000000
501,ART0005111,64.000000,64.00,0.000000
502,ART0005112,104.000000,104.00,0.000000


In [ ]:
tempPrezzo = sqldf('select sum(scostamento) from scostamentoConsumiArtPrezzo')
tempPrezzo

,sum(scostamento)
0,241086.237312


In [ ]:
tempImpiego+tempPrezzo # scostamento totale remains consistent!!! :) 

,sum(scostamento)
0,-96733.03
